# 🦜⛓️ LangChain + 🐘 WangchanGLM v0.1 demo

This notebook will give you how to use 🦜⛓️ LangChain + 🐘 WangchanGLM v0.1.


This notebook under the [Apache Software License 2.0](https://github.com/PyThaiNLP/pythainlp/blob/dev/LICENSE).


The source code come from [How to create ChatGPT Clone](https://python.langchain.com/en/latest/modules/agents/agent_executors/examples/chatgpt_clone.html).

In [ ]:
#@title Install
!pip install --q torch sentencepiece accelerate transformers pandas requests bitsandbytes langchain faiss-gpu sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:

In [ ]:
#@title Model
name_model = "pythainlp/wangchanglm-7.5B-sft-en-sharded" #@param ["pythainlp/wangchanglm-7.5B-sft-en-sharded","pythainlp/wangchanglm-7.5B-sft-enth-sharded"]

In [ ]:
#@title source code
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Optional, Union, List, Dict, Any
import random
import time
import datetime
import os
import re
import pandas as pd
from tqdm.auto import tqdm
# auth_token = os.environ.get("TOKEN_FROM_SECRET") or True

# name_model = "pythainlp/wangchanglm-7.5B-sft-adapter-merged-sharded"
model = AutoModelForCausalLM.from_pretrained(
    name_model, 
    return_dict=True, 
    load_in_8bit=True ,
    device_map="auto", 
    torch_dtype=torch.float16, 
    offload_folder="./", 
    low_cpu_mem_usage=True,
    # use_auth_token=auth_token # , ,
)
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-7.5B")#, use_auth_token=auth_token)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
#@title Install and Import Required Modules
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

from langchain import ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
import torch

from transformers import AutoTokenizer,AutoModelForCausalLM

template = """
{history}
<human>: {human_input}
<bot>:"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)
hf_pipeline = HuggingFacePipeline(pipeline=pipe)

chatgpt_chain = LLMChain(
    llm=hf_pipeline, 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationBufferWindowMemory(k=2),
)


In [ ]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_message = chatgpt_chain.predict(human_input=message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>